# Polarizability by linear response

We compute the polarizability of a Helium atom. The polarizability
is defined as the change in dipole moment
$$
μ = ∫ r ρ(r) dr
$$
with respect to a small uniform electric field $E = -x$.

We compute this in two ways: first by finite differences (applying a
finite electric field), then by linear response. Note that DFTK is
not really adapted to isolated atoms because it uses periodic
boundary conditions. Nevertheless we can simply embed the Helium
atom in a large enough box (although this is computationally wasteful).

As in other tests, this is not fully converged, convergence
parameters were simply selected for fast execution on CI,

In [1]:
using DFTK
using LinearAlgebra

a = 10.
lattice = a * I(3)  # cube of $a$ bohrs
# Helium at the center of the box
atoms     = [ElementPsp(:He, load_psp("hgh/lda/He-q2"))]
positions = [[1/2, 1/2, 1/2]]


kgrid = [1, 1, 1]  # no k-point sampling for an isolated system
Ecut = 30
tol = 1e-8

# dipole moment of a given density (assuming the current geometry)
function dipole(basis, ρ)
    rr = [(r[1] - a/2) for r in r_vectors_cart(basis)]
    sum(rr .* ρ) * basis.dvol
end;

## Using finite differences
We first compute the polarizability by finite differences.
First compute the dipole moment at rest:

In [2]:
model = model_DFT(lattice, atoms, positions;
                  functionals=LDA(), symmetries=false)
basis = PlaneWaveBasis(model; Ecut, kgrid)
res   = self_consistent_field(basis; tol)
μref  = dipole(basis, res.ρ)

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770391569162                   -0.52    9.0    175ms
  2   -2.771690534254       -2.89       -1.32    1.0    137ms
  3   -2.771714290937       -4.62       -2.45    1.0    463ms
  4   -2.771714629162       -6.47       -3.13    1.0   95.1ms
  5   -2.771714713569       -7.07       -3.89    2.0    144ms
  6   -2.771714715038       -8.83       -4.46    1.0    123ms
  7   -2.771714715250       -9.67       -5.75    2.0    113ms
  8   -2.771714715249   +  -12.38       -5.64    2.0    118ms
  9   -2.771714715250      -12.26       -6.93    1.0    102ms
 10   -2.771714715250   +    -Inf       -7.49    2.0    117ms
 11   -2.771714715250      -14.45       -8.41    1.0    113ms


-0.0001345735735031157

Then in a small uniform field:

In [3]:
ε = .01
model_ε = model_DFT(lattice, atoms, positions;
                    functionals=LDA(),
                    extra_terms=[ExternalFromReal(r -> -ε * (r[1] - a/2))],
                    symmetries=false)
basis_ε = PlaneWaveBasis(model_ε; Ecut, kgrid)
res_ε   = self_consistent_field(basis_ε; tol)
με = dipole(basis_ε, res_ε.ρ)

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770449262230                   -0.53    8.0    212ms
  2   -2.771774606407       -2.88       -1.31    1.0    159ms
  3   -2.771801724711       -4.57       -2.55    1.0    100ms
  4   -2.771802030963       -6.51       -3.41    1.0   97.8ms
  5   -2.771802072651       -7.38       -3.88    2.0    567ms
  6   -2.771802074215       -8.81       -4.40    1.0    124ms
  7   -2.771802074466       -9.60       -5.05    1.0    138ms
  8   -2.771802074474      -11.07       -5.60    1.0    129ms
  9   -2.771802074476      -11.76       -6.03    2.0    116ms
 10   -2.771802074476      -12.94       -6.61    1.0    105ms
 11   -2.771802074476      -13.54       -7.09    1.0    104ms
 12   -2.771802074476      -15.05       -7.45    1.0    105ms
 13   -2.771802074476   +  -14.15       -8.11    2.0    133ms


0.01761222136116406

In [4]:
polarizability = (με - μref) / ε

println("Reference dipole:  $μref")
println("Displaced dipole:  $με")
println("Polarizability :   $polarizability")

Reference dipole:  -0.0001345735735031157
Displaced dipole:  0.01761222136116406
Polarizability :   1.7746794934667176


The result on more converged grids is very close to published results.
For example [DOI 10.1039/C8CP03569E](https://doi.org/10.1039/C8CP03569E)
quotes **1.65** with LSDA and **1.38** with CCSD(T).

## Using linear response
Now we use linear response to compute this analytically; we refer to standard
textbooks for the formalism. In the following, $χ_0$ is the
independent-particle polarizability, and $K$ the
Hartree-exchange-correlation kernel. We denote with $δV_{\rm ext}$ an external
perturbing potential (like in this case the uniform electric field). Then:
$$
δρ = χ_0 δV = χ_0 (δV_{\rm ext} + K δρ),
$$
which implies
$$
δρ = (1-χ_0 K)^{-1} χ_0 δV_{\rm ext}.
$$
From this we identify the polarizability operator to be $χ = (1-χ_0 K)^{-1} χ_0$.
Numerically, we apply $χ$ to $δV = -x$ by solving a linear equation
(the Dyson equation) iteratively.

In [5]:
using KrylovKit

# Apply $(1- χ_0 K)$
function dielectric_operator(δρ)
    δV = apply_kernel(basis, δρ; res.ρ)
    χ0δV = apply_χ0(res, δV)
    δρ - χ0δV
end

# `δVext` is the potential from a uniform field interacting with the dielectric dipole
# of the density.
δVext = [-(r[1] - a/2) for r in r_vectors_cart(basis)]
δVext = cat(δVext; dims=4)

# Apply $χ_0$ once to get non-interacting dipole
δρ_nointeract = apply_χ0(res, δVext)

# Solve Dyson equation to get interacting dipole
δρ = linsolve(dielectric_operator, δρ_nointeract, verbosity=3)[1]

println("Non-interacting polarizability: $(dipole(basis, δρ_nointeract))")
println("Interacting polarizability:     $(dipole(basis, δρ))")

[ Info: GMRES linsolve in iter 1; step 1: normres = 2.493920921553e-01
[ Info: GMRES linsolve in iter 1; step 2: normres = 3.766553430670e-03
[ Info: GMRES linsolve in iter 1; step 3: normres = 2.852760178363e-04
[ Info: GMRES linsolve in iter 1; step 4: normres = 4.694609179408e-06
[ Info: GMRES linsolve in iter 1; step 5: normres = 1.088789605212e-08
[ Info: GMRES linsolve in iter 1; step 6: normres = 6.319312334366e-11
[ Info: GMRES linsolve in iter 1; step 7: normres = 1.743517414376e-12
[ Info: GMRES linsolve in iter 1; finished at step 7: normres = 1.743517414376e-12
[ Info: GMRES linsolve in iter 2; step 1: normres = 8.667445480519e-10
[ Info: GMRES linsolve in iter 2; step 2: normres = 8.975821255313e-11
[ Info: GMRES linsolve in iter 2; step 3: normres = 1.650111426077e-12
[ Info: GMRES linsolve in iter 2; finished at step 3: normres = 1.650111426077e-12
┌ Info: GMRES linsolve converged at iteration 2, step 3:
│ *  norm of residual = 1.650114851922771e-12
└ *  number of operat

As expected, the interacting polarizability matches the finite difference
result. The non-interacting polarizability is higher.